<a href="https://colab.research.google.com/github/leonardoub/SCRIPT_PALERMO/blob/master/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>